In [1]:
from mccn.client import MCCN
import pandas as pd

# ENDPOINT = "http://203.101.230.81:8082"
ENDPOINT = "generated/collection.json"

In [91]:
engine = MCCN(ENDPOINT, "llara_campey_point", 100, use_z=True, interp=None)
geobox = engine.geobox
type(geobox.coords)
coords = pd.DataFrame(
    {
        "latitude": geobox.coords["latitude"].values,
        "longitude": geobox.coords["longitude"].values,
    }
)
df = pd.read_csv(engine.point_loader.items[0].location)
df.head()

,latitude,longitude,moisture,elevation_m,yield_t_ha
0,-30.263928,149.860396,0.0,245.60784,2.816659
1,-30.263928,149.860395,0.0,245.57736,0.193452
2,-30.263926,149.860391,0.0,245.57736,0.547192
3,-30.263924,149.860387,0.0,245.57736,0.670937
4,-30.263921,149.860382,0.0,245.57736,0.599533


In [89]:
import duckdb

conn = duckdb.connect()
conn.register("coords", coords)
conn.register("data_table", df)

In [96]:
query = """
WITH distances AS (
    SELECT
        *,
        coords.latitude as x1,
        coords.longitude as y1,
        data_table.latitude as x2,
        data_table.longitude as y2,
        data_table.elevation_m as z,
        sqrt(pow(x1 - x2, 2) + pow(y1 - y2, 2)) AS distance
    FROM coords
    CROSS JOIN data_table
)
SELECT
    distances.x1 as x,
    distances.y1 as y, 
    distances.z as z,
    avg(moisture) as mean_moisture,
    avg(yield_t_ha) as mean_yield
FROM distances where distance <= ?
GROUP BY x, y, z
"""

# Execute query and fetch results
result = conn.execute(query, [2]).fetchdf()
result

,x,y,z,mean_moisture,mean_yield
0,-30.255395,149.859562,245.60784,9.541667,0.915036
1,-30.255395,149.859562,245.24208,10.443750,1.404535
2,-30.255395,149.859562,244.66296,10.852273,1.142728
3,-30.255395,149.859562,244.11432,10.166667,0.431707
4,-30.255660,149.859988,245.15064,10.800000,1.329794
...,...,...,...,...,...
6395,-30.281595,149.901733,244.51056,10.940426,0.693979
6396,-30.281595,149.901733,244.38864,10.222581,0.477046
6397,-30.281595,149.901733,244.14480,10.110526,0.342836
6398,-30.281595,149.901733,245.82120,11.177778,1.757664


In [104]:
from dataclasses import dataclass


@dataclass
class User:
    name: str
    email: str


class UserBuilder(dict):
    def set_name(self, name):
        self["name"] = name
        return self

    def set_email(self, email):
        self["email"] = email
        return self

    def build(self):
        return User(**self)

In [105]:
builder = UserBuilder()

In [106]:
builder.set_name("Son").set_email("lehoangsonsg@gmail.com")

{'name': 'Son', 'email': 'lehoangsonsg@gmail.com'}

In [108]:
builder.build()

User(name='Son', email='lehoangsonsg@gmail.com')

In [109]:
from enum import Enum


class EE(Enum):
    a = "A"
    b = "B"

In [127]:
EE["a"]

<EE.a: 'A'>